In [1]:
import tushare as ts

In [4]:
from sqlalchemy import create_engine

tushare_db = create_engine('postgresql://earthson@localhost/trading_tushare', pool_size=32, max_overflow=0)

In [1]:
def fetch_stock_basics():
    df = ts.get_stock_basics()
    df.to_sql('stock_basics', tushare_db, if_exists="replace")

def fetch_tick(stock, date):
    df = ts.get_tick_data(stock, date=date)
    df['stock'] = stock
    df['date'] = date
    ans = df.set_index(['stock', 'date'])
    try:
        tushare_db.execute("""delete from tick_data where "stock"='%s' AND "date"='%s' """ % (stock, date))
    except:
        pass
    ans.to_sql('tick_data', tushare_db, if_exists="append")

In [2]:
#History data forward answer authority
def fetch_history_faa(stock, start, end):
    df = ts.get_h_data(stock, autype='hfq', start=start, end=end)
    df['stock'] = stock
    ans = df.set_index(['stock','date'])
    try:
        tushare_db.execute("""delete from history_faa where "stock"='%s' AND "date">='%s' AND "date"<='%s'""" % (stock, start, end))
    except:
        pass
    ans.to_sql('history_faa', tushare_db, if_exists="append")

In [81]:
df = ts.get_tick_data('002218', date='2008-03-24')
df['stock'] = '002218'

In [20]:
from datetime import datetime
time_fmt = '%Y-%m-%d %H:%M:%S'

In [5]:
import pandas as pd

In [84]:
df['time'] = ('2008-03-24 ' + df['time']).map(lambda x: pd.Timestamp(datetime.strptime(x, time_fmt), tz='Asia/Shanghai').strftime(format="%Y-%m-%d %H:%M:%S%z"))

In [88]:
bb = df.set_index(['stock', 'time'])

In [89]:
from sqlalchemy import create_engine
from sqlalchemy import TIMESTAMP
tushare_db = create_engine("postgresql+psycopg2://earthson@localhost/trading_tushare", pool_size=16, max_overflow=0)

In [91]:

bb.to_sql('tick_data', tushare_db, dtype={'time': TIMESTAMP(timezone=True)})

In [95]:
hdf = ts.get_h_data('600654', autype='hfq', start='1991-09-01', end='1992-05-14').reset_index()

[Getting data:]###